### 한국어 코퍼스 전처리
- 데이터셋 : Korpora에서 로딩
- 형태소 분석기 설정
- 단어사전 생성

[1] 모듈 로딩 및 데이터 준비 <hr>

In [1]:
# 모듈 로딩
from Korpora import Korpora                         # Open Korean Dataset
from konlpy.tag import *                            # 형태소 분석기
import scipy                                        # 형태소 분석기
import pandas as pd

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import *
from torch.utils.data import Dataset, DataLoader

In [2]:
# 데이터 로딩
nsmc = Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /Users/anhyojun/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /Users/anhy

In [3]:
# Test 데이터셋을 DataFrame으로 로딩
nsmcDF = pd.DataFrame(nsmc.test)
nsmcDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [4]:
nsmcDF = nsmcDF.iloc[:1000,:]
nsmcDF

,text,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...
995,긴박하다. 사람이 저래서 자살을 택하는 것이란걸 알았다. 그 무게를 견디고 스스로 ...,1
996,"어색,유치. 소재는 좋았지만 결정적으로 재미가 없으니..",0
997,"디젤, 개똥폼만잡고 디아블로란 존재도 멋지지도않고",0
998,밋밋하다고밖에 할말이 없다,0


[2] 커스텀 데이터셋 정의 <hr>

In [5]:
# 단어사전 생성 시에 활용
class TextDataset(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label = label
        self.length = feature.shape[0]

    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        return self.feature[index], self.label[index]

In [6]:
nsmcDF

,text,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...
995,긴박하다. 사람이 저래서 자살을 택하는 것이란걸 알았다. 그 무게를 견디고 스스로 ...,1
996,"어색,유치. 소재는 좋았지만 결정적으로 재미가 없으니..",0
997,"디젤, 개똥폼만잡고 디아블로란 존재도 멋지지도않고",0
998,밋밋하다고밖에 할말이 없다,0


In [7]:
# NSMC의 데이터셋 인스턴스 생성
nsmcDS = TextDataset(nsmcDF['text'], nsmcDF['label'])

In [8]:
for f, l in nsmcDS:
    print(f, l)
    break

굳 ㅋ 1


[3] 텍스트 데이터 전처리 <hr>
- 토큰화/정제 (불용어, 구두점, 오타...)
- 단어사전 생성
- 문장 ==> 수치화

In [9]:
# 토크나이즈 생성
okt_tokenizer = get_tokenizer(Okt().morphs)
spacy_tokenizer = get_tokenizer('spacy', language='ko_core_news_sm')

In [10]:
# 텍스트 Dataset에서 처리할 데이터셋만 로딩해서 토큰화 진행하는 반복자 함수
def generateToken(dataset):
    for index in range(len(dataset)):
        text, label = dataset[index]
        yield okt_tokenizer(text)

In [13]:
from torchtext.vocab import build_vocab_from_iterator

# 단어 사전 생성
nsmcVocab = build_vocab_from_iterator(iterator=generateToken(nsmcDS))
nsmcVocab.set_default_index(nsmcVocab['<UNK>'])  # <UNK> 토큰의 인덱스를 기본값으로 설정

1000lines [00:01, 955.84lines/s]


AttributeError: 'Vocab' object has no attribute 'set_default_index'